# **Project No: 2 Email Auto-Drafter using Hugging Face (Text Generation)**

In [2]:
from transformers import pipeline

In [9]:
!pip install transformers torch sentencepiece


In [10]:
from transformers import pipeline

# Load instruction-following model
email_generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-small"
)

# Bullet points input
points = """
Sick leave
Monday
Will return on Tuesday
"""

# Instruction prompt
prompt = f"""
Write a short, polite professional email using the following points:
{points}
"""

# Generate email
result = email_generator(
    prompt,
    max_new_tokens=120
)

# Output
print("Generated Email:\n")
print(result[0]['generated_text'])


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Generated Email:

Sick leave on Monday. Will return on Tuesday.


1.Project Number

Project – 2

2.Aim

To automatically generate a professional email from user-provided bullet points using NLP.

3.Tool Used

Python

Hugging Face Transformers

4.Model Used

distilgpt2

5.Concept Used

Text Generation (NLP)

6.Platform

Google Colab

This project implements an Email Auto-Drafter using Natural Language Processing (NLP) techniques. The system takes user-provided bullet points such as the reason for leave and return date as input and automatically generates a short, polite, and professional email. It uses a pre-trained instruction-based transformer model from Hugging Face to understand the intent of the input text and convert it into a meaningful email message. This project helps reduce manual effort in writing formal emails and demonstrates the practical application of text-to-text generation in real-world productivity tools. The entire system is implemented in Python using Google Colab, making it simple, efficient, and easy to deploy.